In [ ]:
import os
import sys
import pandas as pd
import numpy as np
import pylab as plt
import torch
import torch.nn
from scipy.spatial.transform import Rotation as R

from lyft_dataset_sdk.lyftdataset import LyftDataset
from lyft_dataset_sdk.utils.data_classes import LidarPointCloud, Box, Quaternion
from lyft_dataset_sdk.utils.geometry_utils import view_points, transform_matrix

from utils import utils
from utils.iou.box_overlaps import bbox_overlaps
from config import config as cfg
from data_loader import get_dataloader
from models import VoxelNet

In [ ]:
score_threshold = 0.999
collision_iou_threshold = 0.2
pretrained_model = 'model-car-v3.ckpt'
device = torch.device('cuda:0')

In [ ]:
data_loaders, data_sizes = get_dataloader(phases=['check'])
model = VoxelNet().to(device)
checkpoint = torch.load(os.path.join(cfg.work_dir, 'data/models/pretrain', pretrained_model))
model.load_state_dict(checkpoint['state_dict'])
model.eval()
df = pd.read_csv(os.path.join(cfg.work_dir, 'data/check.csv'))
sample_token = df['sample_token'][0]
sample_token

In [ ]:
voxel_features, voxel_coords, pos_equal_one, neg_equal_one, targets, _ = next(iter(data_loaders['check'])) 

In [ ]:
voxel_features = voxel_features.to(device)
voxel_coords = voxel_coords.to(device)
    
# psm: [batch_size, R_z, H_map, W_map]
# rm: [batch_size, R_z * B_encode, H_map, W_map]
psm, rm = model(voxel_features, voxel_coords, device)
batch_size = psm.size(0)

# psm: [batch_size, H_map, W_map, R_z]
psm = torch.sigmoid(psm.permute(0,2,3,1))

# psm: [batch_size, H_map * W_map * R_z]
psm = psm.reshape((cfg.batch_size, -1))

# rm: [batch_size, H_map, W_map, R_z * B_encode]
rm = rm.permute(0,2,3,1).contiguous()

# rm: [batch_size, H_map, W_map, R_z, R_z * B_encode]
rm = rm.view(rm.size(0), rm.size(1), rm.size(2), 14)

# prob: [batch_size, H_map * W_map * R_z]
prob = psm.view(batch_size, -1)
    
# batch_boxes3d: [batch_size, H_map * W_map * R_z, B_encode]
batch_boxes3d = utils.delta_to_boxes3d(rm, device)

mask = torch.gt(prob, score_threshold)

mask_reg = mask.unsqueeze(2).repeat(1, 1, 7)

batch_id = 0

# boxes3d: [H_map * W_map * R_z, B_encode]
boxes3d = torch.masked_select(batch_boxes3d[batch_id], mask_reg[batch_id]).view(-1, 7)
scores = torch.masked_select(prob[batch_id], mask[batch_id])

# boxes2d_corners: [H_map * W_map * R_z, 4 corners, 2 xy]
boxes3d = boxes3d.cpu().detach().numpy()
boxes2d_corners = utils.boxes3d_to_corners(boxes3d)

print("Predicted 2-D Anchor Boxes:", boxes2d_corners.shape)

In [ ]:
boxes2d = utils.boxes2d_four_corners_to_two_corners(boxes2d_corners)

iou = bbox_overlaps(np.ascontiguousarray(boxes2d).astype(np.float32),
                    np.ascontiguousarray(boxes2d).astype(np.float32))

scores = scores.cpu().detach().numpy()
filter_idc = np.argmax((iou > collision_iou_threshold) * scores, axis=1)
filter_idc = np.unique(filter_idc)
boxes2d_corners = boxes2d_corners[filter_idc]
print("Filtered 2-D Anchor Boxes:", boxes2d_corners.shape)

In [ ]:
lyft_dataset = data_loaders['check'].dataset.lyft_dataset
sample = lyft_dataset.get('sample', sample_token)
lidar_info = lyft_dataset.get('sample_data', sample['data']['LIDAR_TOP'])
lidar_data_path = lyft_dataset.get_sample_data_path(sample['data']['LIDAR_TOP'])
gt_boxes3d = lyft_dataset.get_boxes(sample['data']['LIDAR_TOP'])

ego_pose = lyft_dataset.get('ego_pose', lidar_info['ego_pose_token'])
calibrated_sensor = lyft_dataset.get('calibrated_sensor', lidar_info['calibrated_sensor_token'])

pointclouds = LidarPointCloud.from_file(lidar_data_path)
pointclouds = pointclouds.points.transpose(1, 0)
    
gt_boxes3d = utils.convert_boxes3d_from_global_to_sensor_frame(gt_boxes3d, ego_pose, calibrated_sensor)
    
pointclouds, gt_boxes3d = utils.filter_pointclouds_gt_boxes3d(pointclouds, gt_boxes3d, cfg.class_name)    

gt_boxes3d_xyzlwhr = np.array([[gt_box3d.center[0],
                                gt_box3d.center[1],
                                gt_box3d.center[2],
                                gt_box3d.wlh[1],
                                gt_box3d.wlh[0],
                                gt_box3d.wlh[2],
                                gt_box3d.orientation.yaw_pitch_roll[0]] for gt_box3d in gt_boxes3d])

gt_boxes2d_corners = utils.gt_boxes3d_center_to_bottom_corner(gt_boxes3d)

In [ ]:
fig, ax = plt.subplots(figsize=(40, 40))

 
ax.plot(gt_boxes2d_corners[:, :, 0].astype(np.float32), gt_boxes2d_corners[:, :, 1].astype(np.float32),
        'X', markersize=20)

ax.plot(boxes2d_corners[:, :, 0].astype(np.float32), boxes2d_corners[:, :, 1].astype(np.float32),
        'o', markersize=10)
ax.set_xlim([-80, 80])
ax.set_ylim([-80, 80])

In [ ]:
fig, ax = plt.subplots(figsize=(40, 40))

ax.plot(pointclouds[:, 0], pointclouds[:, 1], 'o')
ax.plot(gt_boxes2d_corners[:, :, 0].astype(np.float32), gt_boxes2d_corners[:, :, 1].astype(np.float32),
        'X', markersize=20)
ax.plot(boxes2d_corners[:, :, 0].astype(np.float32), boxes2d_corners[:, :, 1].astype(np.float32),
        'o', markersize=10)

ax.set_xlim([-80, 80])
ax.set_ylim([-80, 80])

In [ ]:
fig, ax = plt.subplots(figsize=(40, 40))
ax.plot(gt_boxes2d_corners[:, :, 0].astype(np.float32), gt_boxes2d_corners[:, :, 1].astype(np.float32),
        'X', markersize=20)

ax.set_xlim([-80, 80])
ax.set_ylim([-80, 80])

In [ ]:
fig, ax = plt.subplots(figsize=(40, 40))

ax.plot(boxes2d_corners[:, :, 0].astype(np.float32), boxes2d_corners[:, :, 1].astype(np.float32),
        'o', markersize=10) 

ax.set_xlim([-80, 80])
ax.set_ylim([-80, 80])